In [ ]:
from Recommenders.ScoresHybridRecommender import ScoresHybridRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Utils.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from Utils import Reader
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
#ALL TRIES
import numpy
import itertools
from tqdm.notebook import tqdm
# GRID SEARCH RP3BETA

URM_all=Reader.read_train_csr(values_to_replace={0:0.03})
#URM_all=Reader.load_URM("../output/URM_2_mean.csv")
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all ,train_percentage=0.80)
evaluator_test=EvaluatorHoldout(URM_test,cutoff_list=[10])
ials=IALSRecommender(URM_train)
ials.fit( num_factors= 77, epochs= 115, confidence_scaling= 'linear', alpha=  3.951833188581218, epsilon= 1.1320191944038327, reg= 4.746687709282205e-05)
#recommender_ItemKNNCF = ItemKNNCFRecommender(URM_train)
#recommender_ItemKNNCF.fit()
rp3=RP3betaRecommender(URM_train)
rp3.fit(
    topK= 339, alpha= 0.0, beta= 0.3326953675715214, normalize_similarity= True
)
pureSVD=PureSVDRecommender(URM_train)
pureSVD.fit(
    num_factors= 30
)

P3alpha=P3alphaRecommender(URM_train)
P3alpha.fit(
    topK= 564, alpha= 0.18769516845528106, normalize_similarity= True
)

algoList=[pureSVD,P3alpha,rp3]

best=0
alpha=0
algo=""
dList=numpy.arange(0.25, 1, 0.25)
for x in tqdm(itertools.product(dList,algoList),total=int(len(dList)*len(algoList)), position=0, leave=True):

    scoreshybridrecommender = ScoresHybridRecommender(URM_train, ials, x[1])
    scoreshybridrecommender.fit(alpha = x[0])
    result_df, _ = evaluator_test.evaluateRecommender(scoreshybridrecommender)


    if result_df["MAP"][10] > best:
        best=result_df["MAP"][10]
        alpha=x[0]
        algo=x[1]

